In [1]:
import numpy as np
import pandas as pd
import os
import pickle
import re
import calendar

In [2]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_colwidth', 500)
pd.options.display.float_format = '{:.0f}'.format

In [3]:
DATA_ROOT_PATH="/mnt/data/projects/MD1/data/R1000/reports/"
PROJ_ROOT_PATH="/home/priyesh/projects/MD1"

In [4]:
# Read from Pickle file

filepath = os.path.join(PROJ_ROOT_PATH,'pickle','clean_master_data_stage2.pkl')
clean_master_data = pd.read_pickle(filepath)

In [5]:
# Read from Pickle file

table_labels = {}

filepath = os.path.join(PROJ_ROOT_PATH,'pickle','mappings_cash_stage3.pkl')
table_labels['cash'] = pd.read_pickle(filepath)
                        
filepath = os.path.join(PROJ_ROOT_PATH,'pickle','mappings_bal_stage3.pkl')
table_labels['balance'] = pd.read_pickle(filepath)
                        
filepath = os.path.join(PROJ_ROOT_PATH,'pickle','mappings_income_stage3.pkl')
table_labels['income'] = pd.read_pickle(filepath)

## Create Lookups

In [6]:
heading_lookup = {}
line_item_lookup = {}

for stype in ['balance','income','cash']:
    
  temp = table_labels[stype]['headings'].to_dict()
  heading_lookup[stype] = temp['type']

  temp = table_labels[stype]['line_items'].to_dict()
  line_item_lookup[stype] = temp['type']

In [7]:
heading_lookup['income']['Operating Revenues:']

'SR'

In [8]:
# Create lookup dictionaries

swap_text_dict = {}
swap_text_dict['income'] = {}
swap_text_dict['cash'] = {}
swap_text_dict['balance'] = {}
              
swap_text_dict['income']['H'] = { 'Net Sales:' : 'Revenue:' }
swap_text_dict['income']['L'] = {}

swap_text_dict['cash']['H'] = {}
swap_text_dict['cash']['L'] = {}

swap_text_dict['balance']['H'] = {}
swap_text_dict['balance']['L'] = {}


## Create standardised table

In [9]:
def swap_text(type, text, stype):
  
  # Default to original text. Swap if found in lookup.
    
  new_text = text
    
  if text in swap_text_dict[stype][type]:
      
    new_text = swap_text_dict[stype][type][text]

  return new_text      


In [10]:
def find_line_items(table, heading_lookup, line_item_lookup, stype):
 
  df_new = pd.DataFrame()

  h = 'No Heading'
  items_found = []
    
  h_lookup = heading_lookup[stype]
  l_lookup = line_item_lookup[stype]
    
  items_required = list(set(l_lookup.values()))

  # Go through each line in table, if row contains only nan then assume heading

  for index, row in table.iterrows():

    if row.isnull().all():
   
      # If all columns are null then it is either a header we are interested in, a header not recognised 
      # or a line item where values are not specified.
    
      # If we encounter a header that has Member or Abstract then terminate loop and ignore subsequent items.
    
      if index in h_lookup:
        
        h = h_lookup[index]
        
        if h == 'EXCL':
          print(index)
          break
        else:
          r1 = row.copy()
          r1 = r1.rename(swap_text('H',index,stype))
          r1['type'] = h
          r1['line_type'] = 'H'
          df_new = pd.concat([df_new, pd.DataFrame([r1])])
  
    else:
      # If row matches one of the items mapped for current header then store
    
      if index in l_lookup:
        
        l = l_lookup[index]
        items_found.append(l)
          
        r1 = row.copy()
        r1.rename(swap_text('L',index,stype))
        r1['type'] = l
        r1['line_type'] = 'L'
        df_new = pd.concat([df_new, pd.DataFrame([r1])])

  return(df_new)

In [11]:
df = clean_master_data['AAL']['18']['income']['table']
df

,"Dec. 31, 2017","Dec. 31, 2016","Dec. 31, 2015"
line_item,,,
Operating Revenues:,NaN,NaN,NaN
Mainline Passenger,29238,27909,29037
Regional Passenger,6895,6670,6475
Cargo,800,700,760
Other,5274,4901,4718
Total Operating Revenue,42207,40180,40990
Operating Expenses:,NaN,NaN,NaN
Aircraft Fuel And Related Taxes,6128,5071,6226
"Salaries, Wages And Benefits",11816,10890,9524


In [12]:
find_line_items(df, heading_lookup, line_item_lookup,'income')

Earnings Per Common Share:


,"Dec. 31, 2017","Dec. 31, 2016","Dec. 31, 2015",type,line_type
Operating Revenues:,NaN,NaN,NaN,SR,H
Total Operating Revenue,42207,40180,40990,R,L
Operating Expenses:,NaN,NaN,NaN,OE,H
Total Operating Expenses,38149,34896,34786,TOE,L
Operating Income,4058,5284,6204,OI,L
Net Income,1919,2676,7610,NI,L


In [13]:
heading_lookup['income']['Cost Of Sales:']

'COS'

In [13]:
def conv_date_cols(cols):
    
  mnth_lookup = {month: index for index, month in enumerate(calendar.month_abbr) if month}

  new_cols = []

  for c in cols:
    
    x = re.search(r"^[A-Z][a-z][a-z]\. \d{2}, \d{4}", c)
    
    if x:
      mnth = mnth_lookup[c.split(".")[0]]
      yr = c[-4:]
      t = yr + '-' + str(mnth)
      new_cols = new_cols + [t]
    else:
      new_cols = new_cols + [c] 

  return new_cols

In [14]:
def extract_statements(stype):
    
  ticker_list = list(clean_master_data.keys())

  master_statements = {}

  for ticker in ticker_list:
    
    statement = {}
    
    for yr in clean_master_data[ticker]:
    
      # Extract statement for each year
    
      if stype in clean_master_data[ticker][yr]:
        
        statement[yr] = find_line_items(clean_master_data[ticker][yr][stype]['table'],
                                               heading_lookup, line_item_lookup,stype)
    
    # If number of keys is 0 then we didn't find any statements for the give ticker 
    # so skip the construction of full table.
    
    keys = list(statement.keys())

    if len(keys) > 0:
    
      print(ticker)
      # Take the first statement and then iterate through the rest, merging each 
      # one to build the complete table.
      # Only take columns from the next statement where there is no overlap.

      keys = sorted(keys,reverse=True)
      df = statement[keys[0]].copy()
      df = df[~df.index.duplicated(keep='first')]

      for i in range(1,len(keys)):
        
        df1_cols = list(set(statement[keys[i]].columns)- set(df.columns))
        df1 = statement[keys[i]][df1_cols].copy()
        df1 = df1[~df1.index.duplicated(keep='first')] 
        df = df.merge(df1, right_index=True, left_index=True, how='left', suffixes=['_x', '_y'])
       
    
      # We could end up with an empty dataframe if there are no matches. So skip if that is the case.
     
      if len(df) > 0:
        # Amend date format of column names
      
        cols = list(df.columns)
        new_cols = conv_date_cols(cols)
        df.columns = new_cols

        # Sort in reverse date order, place type and line_type at end
    
        new_cols.remove('type')
        new_cols.remove('line_type')
        cols = sorted(new_cols,reverse=True) + ['type','line_type']
        df = df[cols]
        
        master_statements[ticker] = df

  return master_statements


In [17]:
master_std_statements = {}

In [18]:
master_std_statements['cash'] = extract_statements('cash')


AA
AAL
AAP
AAPL
ABBV
ABNB
ABT
ACGL
ACHC
ACI
ACM
ACN
ADBE
ADI
ADM
ADP
ADSK
ADT
AEE
AEP
AES
AFG
AFL
AFRM
AGCO
AGL
AGNC
AGO
AGR
AIG
AIZ
AJG
AKAM
AL
ALB
ALGM
ALGN
ALK
ALL
ALLE
ALLY
ALNY
ALSN
AM
AMAT
AMC
AMCR
AMD
AME
AMED
AMG
AMGN
AMH
AMP
AMT
AMZN
AN
ANET
ANSS
AON
AOS
APA
APD
APH
APO
APP
APTV
AR
ARE
ARES
ARMK
ARW
ASH
ATO
ATR
ATUS
ATVI
AVB
AVGO
AVT
AVTR
AWI
AWK
AXON
AXS
AXTA
AYI
AYX
AZEK
AZO
AZTA
BA
BAC
BAH
BALL
BAX
BBWI
BBY
BC
BDX
BEN
BERY
BFAM
BG
BHF
BIIB
BILL
BIO
BJ
BK
BKI
BKNG
BLD
BLDR
BLK
BMRN
BMY
BOKF
BR
BRKR
BRO
BRX
BSX
BSY
BURL
BWA
BWXT
BX
BXP
BYD
C
CABO
CACC
CACI
CAG
CAH
CAR
CARR
CASY
CAT
CB
CBOE
CBRE
CBSH
CC
CCCS
CCI
CCK
CCL
CDAY
CDNS
CDW
CE
CEG
CERT
CF
CFG
CFLT
CFR
CG
CGNX
CHD
CHDN
CHE
CHH
CHK
CHPT
CHRW
CHTR
CI
CIEN
CINF
CL
CLH
CLVT
CLX
CMA
CMCSA
CME
CMG
CMI
CMS
CNA
CNC
CNM
CNP
CNXC
COF
COIN
COLB
COLD
COLM
COO
COST
COTY
CPB
CPRI
CPRT
CPT
CR
CRI
CRL
CRUS
CRWD
CSCO
CSGP
CSL
CSX
CTAS
CTLT
CTRA
CTSH
CTVA
CUBE
CUZ
CVNA
CVS
CVX
CW
CZR
D
DAL
DAR
DASH
DBX
DCI
DD
DDOG
DECK
DEI
DFS
DG
DGX
D

In [17]:
master_std_statements['income'] = extract_statements('income')

Earnings Per Share Attributable To Alcoa Corporation Common Shareholders -F:
Earnings Per Share Attributable To Alcoa Corporation Common Shareholders -F:
Earnings Per Share Attributable To Alcoa Corporation Common Shareholders -F:
Earnings Per Share Attributable To Alcoa Corporation Common Shareholders -F:
Earnings Per Share Attributable To Alcoa Corporation Common Shareholders -F:
Earnings Per Share Attributable To Alcoa Corporation Common Shareholders -F:
Earnings Per Share Attributable To Alcoa Corporation Common Shareholders -F & I:
AA
Earnings Per Common Share:
Earnings -Loss Per Common Share:
Weighted Average Shares Outstanding -In Thousands:
Weighted Average Shares Outstanding -In Thousands
Weighted Average Shares Outstanding -In Thousands:
Earnings -Loss Per Common Share:
Earnings -Loss Per Common Share:
Earnings Per Common Share:
Earnings Per Common Share:
Earnings Per Common Share:
AAL
AAP
Shares Used In Computing Earnings Per Share:
Shares Used In Computing Earnings Per Shar

Net Income -Loss Per Common Share, Basic And Diluted
Net Income -Loss Per Common Share, Basic And Diluted
AGL
Dividends Declared Per Common Share
Dividends Declared Per Common Share
Dividends Declared Per Common Share
Dividends Declared Per Common Share
Dividends Declared Per Common Share
AGNC
AGO
Weighted-Average Number Of Common Shares Outstanding:
Weighted-Average Number Of Common Shares Outstanding:
Weighted-Average Number Of Common Shares Outstanding:
Weighted-Average Number Of Common Shares Outstanding:
Weighted-Average Number Of Common Shares Outstanding:
Weighted-Average Number Of Common Shares Outstanding:
Weighted-Average Number Of Common Shares Outstanding:
Weighted-Average Number Of Common Shares Outstanding:
AGR
Basic:
Basic:
Basic:
Basic:
Basic:
Basic:
Basic:
Basic:
Basic:
Basic And Diluted:
Basic:
AIG
Basic
Basic
Earnings Per Common Share
Earnings Per Common Share
Earnings Per Common Share
AIZ
Basic Net Earnings -Loss Per Share:
Commissions [Member]
Commissions [Member]


Basic Earnings -Loss Per Share:
Basic Earnings -Loss Per Share:
Basic Earnings -Loss Per Share:
Basic Earnings -Loss Per Share:
Basic Earnings -Loss Per Share:
AN
Net Income Attributable To Common Stockholders:
Weighted-Average Common Shares Outstanding
Net Income Attributable To Common Stockholders:
Net Income Attributable To Common Stockholders:
Net Income Attributable To Common Stockholders:
Net Income Attributable To Common Stockholders:
Net Income Attributable To Common Stockholders:
Net Income Attributable To Common Stockholders:
Net Income Attributable To Common Stockholders:
ANET
Earnings Per Share-Basic:
Earnings Per Share – Basic:
Earnings Per Share – Basic:
Earnings Per Share – Basic:
Earnings Per Share – Basic:
Earnings Per Share – Basic:
Earnings Per Share – Basic:
Earnings Per Share – Basic:
Earnings Per Share – Basic:
Earnings Per Share – Basic:
Earnings Per Share – Basic:
Earnings Per Share – Basic:
ANSS
Basic Net Income Per Share Attributable To Aon Shareholders
Basic 

Weighted Average Shares Outstanding:
Weighted Average Shares Outstanding:
Weighted Average Shares Outstanding:
Weighted Average Shares Outstanding:
Net Income Per Common And Common Equivalent Shares:
Weighted Average Shares Outstanding:
AXON
Net Income Per Common Share:
Net Income Per Common Share
Earnings -Loss Per Common Share:
Net Income Per Common Share
Net Income Per Common Share:
Net Income Per Common Share
Earnings -Loss Per Common Share:
Earnings -Loss Per Common Share:
Earnings -Loss Per Common Share:
Net Income -Loss Per Common Share:
Net Income Per Common Share:
Earnings -Loss Per Common Share:
AXS
Successor [Member]
Successor [Member]
AXTA
AYI
AYX
Net Income -Loss Per Common Share:
Net Income -Loss Per Common Share:
Net Income -Loss Per Common Share:
AZEK
AZO
Basic Net Income Per Share:
AZTA
Product [Member]
Product [Member]
Product [Member]
Product [Member]
BA
Per Common Share Information
Per Common Share Information
Per Common Share Information
Per Common Share Informatio

BKNG
Basic:
Net Income Per Common Share:
Basic:
Net Income Per Common Share:
Net Income Per Common Share:
Income Per Common Share:
Basic:
Net Income Per Common Share:
BLD
Basic And Diluted Net Loss Per Share:
Weighted Average Common Shares Outstanding:
Weighted Average Common Shares:
Basic Net Income -Loss Per Share:
Basic Net Income -Loss Per Share:
Weighted Average Common Shares:
Weighted Average Common Shares Outstanding:
Weighted Average Common Shares Outstanding:
Weighted Average Common Shares Outstanding:
Basic And Diluted Net Loss Per Share:
Weighted Average Common Shares Outstanding:
BLDR
Earnings Per Share Attributable To Blackrock, Inc. Common Stockholders:
Earnings Per Share Attributable To Blackrock, Inc. Common Stockholders:
Earnings Per Share Attributable To Blackrock, Inc. Common Stockholders:
Earnings Per Share Attributable To Blackrock, Inc. Common Stockholders:
Earnings Per Share Attributable To Blackrock, Inc. Common Stockholders:
Earnings Per Share Attributable To B

...

SYY
T
Basic Net Income -Loss Attributable To Molson Coors Brewing Company Per Share:
Basic Net Income -Loss Attributable To Molson Coors Brewing Company Per Share:
Basic Net Income -Loss Attributable To Molson Coors Beverage Company Per Share:
Basic Net Income -Loss Attributable To Molson Coors Brewing Company Per Share:
Basic Net Income -Loss Attributable To Molson Coors Brewing Company Per Share:
Basic Net Income -Loss Attributable To Molson Coors Brewing Company Per Share:
Basic Net Income -Loss Attributable To Molson Coors Beverage Company Per Share:
Basic Net Income -Loss Attributable To Molson Coors Beverage Company Per Share:
Basic Net Income -Loss Attributable To Molson Coors Brewing Company Per Share:
Basic Net Income -Loss Attributable To Molson Coors Brewing Company Per Share:
Basic Net Income -Loss Attributable To Molson Coors Brewing Company Per Share:
Basic Net Income -Loss Attributable To Molson Coors Beverage Company Per Share:
TAP
Net Income Per Common Share
Net Income

Earnings Per Common Share:
Basic
TPX
TREX
Sales Of Commodities [Member]
Sales Of Commodities [Member]
Sales Of Commodities [Member]
Sales Of Commodities [Member]
Sales Of Commodities [Member]
TRGP
Weighted Average Common Shares Outstanding:
Earnings Per Share Attributable To Common Stockholders -Note 5:
Earnings -Loss Per Share Attributable To Common Stockholders -Note 16:
Earnings Per Share Attributable To Common Stockholders -Note 2:
Weighted Average Common Shares Outstanding:
Earnings Per Share Attributable To Tripadvisor, Inc. Available To Common Stockholders -Note 2:
Earnings -Loss Per Share Attributable To Common Stockholders -Note 17:
Earnings -Loss Per Share Attributable To Common Stockholders -Note 17:
Earnings -Loss Per Share Attributable To Common Stockholders -Note 5:
Earnings -Loss Per Share Attributable To Common Stockholders -Note 5:
Weighted Average Common Shares Outstanding:
Earnings -Loss Per Share Attributable To Common Stockholders -Note 5:
TRIP
Shares Used In Calcu

UHS
Net Income -Loss Per Share Of Common Stock:
Net Income Per Share Of Common Stock:
Net Income Per Share Of Common Stock:
Net Income -Loss Per Share Of Common Stock:
Net Income Per Share Of Common Stock:
Net Income Per Share Of Common Stock:
Net Income Per Share Of Common Stock:
Net Income Per Share Of Common Stock:
Net Income Per Share Of Common Stock:
Net Income -Loss Per Share Of Common Stock:
Net Income Per Share Of Common Stock:
UI
Net Income Per Common Share:
Net Income Per Common Share:
Net Income Per Common Share:
Net Income Per Common Share:
Net Income Per Common Share:
Net Income Per Common Share:
Net Income Per Common Share:
Net Income Per Common Share:
Net Income Per Common Share:
Net Income Per Common Share:
Net Income Per Common Share:
Net Income Per Common Share:
ULTA
Earnings Per Share Attributable To Unitedhealth Group Common Shareholders:
Earnings Per Share Attributable To Unitedhealth Group Common Shareholders:
Earnings Per Share Attributable To Unitedhealth Group 

VSCO
Weighted Average Shares Of Common Stock Outstanding:
Weighted Average Shares Of Common Stock Outstanding:
Weighted Average Shares Of Common Stock Outstanding:
Weighted Average Shares Of Common Stock Outstanding:
Weighted Average Shares Of Common Stock Outstanding:
Weighted Average Shares Of Common Stock Outstanding:
VST
Basic:
Basic:
Basic:
Basic:
Basic:
Basic:
Basic:
Basic:
Basic:
Basic:
Basic:
Basic:
VTR
Weighted Average Shares Outstanding:
Weighted Average Shares Outstanding:
Weighted Average Shares Outstanding:
VTRS
Weighted Average Common Shares Outstanding
Weighted Average Common Shares Outstanding
Weighted Average Common Shares Outstanding
Weighted Average Common Shares Outstanding
Weighted Average Common Shares Outstanding
VVV
Basic Earnings Per Common Share
Basic Earnings Per Common Share
Basic Earnings Per Common Share
Basic Earnings Per Common Share
Basic Earnings Per Common Share
Basic Earnings Per Common Share
Basic Earnings Per Common Share
Basic Earnings Per Common 

Weighted Average Ordinary Shares Outstanding -Excluding Shares Subject To Possible Redemption:
Earnings Per Share From Continuing Operations Attributable To Willscot Mobile Mini Common Shareholders:
Earnings -Loss Per Share Attributable To Willscot Mobile Mini Common Shareholders:
Earnings -Loss Per Share Attributable To Willscot Mobile Mini Common Shareholders:
Net -Loss Income Per Share Attributable To Willscot Common Shareholders - Basic And Diluted
Net Income -Loss Per Share Attributable To Wsc – Basic And Diluted
-Loss Income Per Share Attributable To Willscot Common Shareholders - Basic And Diluted
WSC
Shares Used In Calculation Of Earnings Per Share:
Shares Used In Calculation Of Earnings Per Share:
Shares Used In Calculation Of Earnings Per Share:
Shares Used In Calculation Of Earnings Per Share:
Shares Used In Calculation Of Earnings Per Share:
Shares Used In Calculation Of Earnings Per Share:
Shares Used In Calculation Of Earnings Per Share:
Shares Used In Calculation Of Earn

In [20]:
master_std_statements['balance'] = extract_statements('balance')

AA
AAL
AAP
AAPL
ABBV
ABNB
ABT
ACGL
ACHC
ACI
ACM
ACN
ADBE
ADI
ADM
ADP
ADSK
ADT
AEE
AEP
AES
AFG
AFL
AFRM
AGCO
AGL
AGNC
AGO
AGR
AIG
AIZ
AJG
AKAM
AL
ALB
ALGM
ALGN
ALK
ALL
ALLE
ALLY
ALNY
ALSN
AM
AMAT
AMC
AMCR
AMD
AME
AMED
AMG
AMGN
AMH
AMP
AMT
AMZN
AN
ANET
ANSS
AON
AOS
APA
APD
APH
APO
APP
APTV
AR
ARE
ARES
ARMK
ARW
ASH
ATO
ATR
ATUS
ATVI
AVB
AVGO
AVT
AVTR
AWI
AWK
AXON
AXS
AXTA
AYI
AYX
AZEK
AZO
AZTA
BA
BAC
BAH
BALL
BAX
BBWI
BBY
BC
BDX
BEN
BERY
BFAM
BG
BHF
BIIB
BILL
BIO
BJ
BK
BKI
BKNG
BLD
BLDR
BLK
BMRN
BMY
BOKF
BR
BRKR
BRO
BRX
BSX
BSY
BURL
BWA
BWXT
BX
BXP
BYD
C
CABO
CACC
CACI
CAG
CAH
CAR
CARR
CASY
CAT
CB
CBOE
CBRE
CBSH
CC
CCI
CCK
CCL
CDAY
CDNS
CDW
CE
CEG
CERT
CF
CFG
CFLT
CFR
CG
CGNX
CHD
CHDN
CHE
CHH
CHK
CHPT
CHRW
CHTR
CI
CIEN
CINF
CL
CLH
CLVT
CLX
CMA
CMCSA
CME
CMG
CMI
CMS
CNA
CNC
CNM
CNP
CNXC
COF
COIN
COLB
COLD
COLM
COO
COST
COTY
CPB
CPRI
CPRT
CPT
CR
CRI
CRL
CRUS
CRWD
CSCO
CSGP
CSL
CSX
CTAS
CTLT
CTRA
CTSH
CTVA
CUBE
CUZ
CVNA
CVS
CVX
CW
CZR
D
DAL
DAR
DASH
DBX
DCI
DD
DDOG
DECK
DEI
DFS
DG
DGX
DH
DHI

In [24]:
master_std_statements['income']['BA']

,2022-12,2021-12,2020-12,2019-12,2018-12,2017-12,2016-12,2015-12,2014-12,2013-12,2012-12,2011-12,2010-12,2009-12,type,line_type
Total Revenue,66608,62286,58158,76559,101127,94005,93496,96114,90762,86623,81698,68735,64306,68281,R,L
Total Costs And Expenses,-63106,-59269,-63843,-72093,-81490,-76612,-79026,-82088,-76752,-73268,-68665,-55888,-51843,-56540,TOE,L
Gross Profit,3502,3017,-5685,4466,19637,17393,14470,14026,14010,13355,13033,12847,12463,11741,GP,L
General And Administrative Expense,-4187,-4157,-4817,-3909,-4567,-4095,-3613,-3525,-3767,-3956,-3717,-3408,-3644,-3364,SGA,L
Cost Of Goods And Services Sold,-53969,-49954,-54568,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CR,L


In [18]:
# Save to file

filepath = os.path.join(PROJ_ROOT_PATH,'pickle','std_statements_stage4.pkl')
with open(filepath, 'wb') as f:
  pickle.dump(master_std_statements, f)


In [16]:
# Read from Pickle file

filepath = os.path.join(PROJ_ROOT_PATH,'pickle','std_statements_stage4.pkl')
master_std_statements = pd.read_pickle(filepath)

In [38]:
df = master_std_statements['income']['WMT']